<a href="https://colab.research.google.com/github/Macgie29/FootballAnalytics/blob/main/Own_Analysis_Polish_passes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing necessary libraries
import pathlib
import os
import pandas as pd
import json


from sklearn.preprocessing import MultiLabelBinarizer

import matplotlib.pyplot as plt
import numpy as np
from mplsoccer import Pitch, Sbopen, VerticalPitch

# New section

In [ ]:
tags = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Football_Analytics/Learning/wyscout/tags2name.csv')
events_name = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Football_Analytics/Learning/wyscout/eventid2name.csv')
events_name.head()

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Football_Analytics/Learning/wyscout/matches/matches_World_Cup.json"

with open(path) as f:
    data = json.load(f)
#save it in a dataframe
df_matches = pd.DataFrame(data)
#structure of data
df_matches.head()

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Football_Analytics/Learning/wyscout/teams.json"

with open(path) as f:
    data = json.load(f)
#save it in a dataframe
df_teams = pd.DataFrame(data)
#structure of data
df_teams.head()

In [ ]:
poland = df_teams[df_teams.wyId == 13869].wyId
poland

In [ ]:
polish_matches = df_matches[df_matches['teamsData'].apply(lambda x: '13869' in x.keys())]
polish_matches_list = list(polish_matches.wyId)
polish_matches


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Football_Analytics/Learning/wyscout/events/events_World_Cup.json"

with open(path) as f:
    data = json.load(f)
#save it in a dataframe
world_cup_events = pd.DataFrame(data)
#structure of data
polish_matches_events = world_cup_events[world_cup_events.matchId.isin(polish_matches_list)]

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Football_Analytics/Learning/wyscout/players.json"

with open(path, encoding='iso8859_2') as f:
    data = json.load(f)
#save it in a dataframe
players_world_cup = pd.DataFrame(data)
#structure of data
polish_players_wc  = players_world_cup[players_world_cup['passportArea'].apply(lambda x: x['name']) == 'Poland']
polish_players_wc.head()

polish_players = polish_players_wc[['shortName', 'wyId', 'foot', 'lastName']].rename(columns={"wyId": "playerId"})

In [ ]:
polish_matches_events

In [ ]:
polish_matches

In [ ]:
polish_matches

# **Analyzing**

In [ ]:
#Data Cleaning

#matches_events
polish_matches_events[['y', 'x', 'end_y', 'end_x']] = polish_matches_events['positions'].apply(lambda x: pd.Series({'y': x[0]['y'], 'x': x[0]['x'], 'end_y': x[1]['y'], 'end_x': x[1]['x']}))
polish_matches_events = polish_matches_events.drop(columns=['positions'], axis=1)
polish_matches_events = polish_matches_events.set_index('id')

#players_info

# extract players info
# extract players info
players_info = []
for match_id, team_data in zip(polish_matches['wyId'], polish_matches['teamsData']):
    for team_id, team_info in team_data.items():
        for player_info in team_info['formation']['lineup']:
            player_info['teamId'] = team_id
            player_info['inFormation'] = True
            player_info['sub'] = False
            player_info['minute'] = None
            player_info['playerIn'] = ''
            player_info['playerOut'] = ''
            player_info['matchId'] = match_id
            players_info.append(player_info)
        for player_info in team_info['formation']['bench']:
            player_info['teamId'] = team_id
            player_info['inFormation'] = False
            player_info['sub'] = False
            player_info['minute'] = None
            player_info['playerIn'] = ''
            player_info['playerOut'] = ''
            player_info['matchId'] = match_id
            players_info.append(player_info)
        for sub_info in team_info['formation']['substitutions']:
            player_in = [p for p in players_info if p['playerId'] == sub_info['playerIn']][0]
            player_out = [p for p in players_info if p['playerId'] == sub_info['playerOut']][0]
            player_out['sub'] = True
            player_out['minute'] = sub_info['minute']
            player_out['playerIn'] = sub_info['playerIn']
            player_out['playerOut'] = sub_info['playerOut']
            player_out['matchId'] = match_id
            player_in['inFormation'] = False
            player_in['sub'] = True
            player_in['minute'] = sub_info['minute']
            player_in['playerIn'] = sub_info['playerIn']
            player_in['playerOut'] = sub_info['playerOut']
            player_in['matchId'] = match_id

# create a DataFrame from the players info
players_df = pd.DataFrame(players_info)

# select the desired columns
players_df = players_df[['matchId', 'teamId', 'playerId', 'inFormation', 'sub', 'minute', 'playerIn', 'playerOut']]

# print the DataFrame
polish_players = pd.merge(players_df, polish_players, on='playerId', how='left')


#matches_info

polish_matches = polish_matches[['wyId', 'label']].rename(columns={"wyId": "matchId"})

polish_matches_events = polish_matches_events.merge(polish_matches, on='matchId', how='left')

# Convert the 'tags' column to a list of tag IDs
polish_matches_events['tags'] = polish_matches_events['tags'].apply(lambda tag_list: [tag['id'] for tag in tag_list])

# One-hot encode the tag IDs using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
tags_dummies = pd.DataFrame(mlb.fit_transform(polish_matches_events['tags']), columns=mlb.classes_, index=polish_matches_events.index)

# Create a mapping of tag IDs to labels
id_to_label = tags.set_index('Tag')['Label'].to_dict()

# Rename the columns of tags_dummies using the id_to_label mapping
tags_dummies.columns = tags_dummies.columns.map(id_to_label)

# Merge the one-hot encoded DataFrame with the original polish_matches_events dataset
polish_matches_events = pd.concat([polish_matches_events, tags_dummies], axis=1)

# Drop the original 'tags' column
polish_matches_events = polish_matches_events.drop(columns=['tags'])

polish_matches_events.head()


In [ ]:
polish_players.head()

In [ ]:
polish_matches_events.head()

Analyzing shots of the polish team in three matches

In [ ]:
polish_players_names = polish_players.loc[:, ['playerId', 'lastName']].drop_duplicates(subset=['playerId', 'lastName'])
polish_players_names

In [ ]:
df = polish_matches_events[(polish_matches_events.eventName=='Shot') & (polish_matches_events.teamId == 13869)].rename(columns={'label': 'matchName'})
df = pd.merge(df, polish_players_names, on='playerId')
df.head()


# Define a custom function to determine the shot outcome
def shot_outcome(row):
    if row['Goal'] == 1:
        return 'Goal'
    elif row['accurate'] == 1:
        return 'Accurate'
    elif row['not accurate'] == 1:
        return 'Missed'

# Apply the custom function to create the 'shot_outcome' column
df['shot_outcome'] = df.apply(shot_outcome, axis=1)

df.head()




In [ ]:
df[['Goal', 'accurate', 'not accurate']]

In [ ]:
for i in df.matchId.unique():
  plot_df = df[(df.matchId == i)]
  pitch = Pitch(line_zorder=2, line_color="black")
  fig, ax = pitch.draw(figsize=(12, 8))
  # Size of the pitch in yards (!!!)
  pitchLengthX = 120
  pitchWidthY = 80
  # Standardize the 'x' and 'y' values
  plot_df['x_standardized'] = plot_df['x'] / 100 * pitchLengthX
  plot_df['y_standardized'] = plot_df['y'] / 100 * pitchWidthY
# Plot the shots by looping through them.
  for i, shot in plot_df.iterrows():
      # Get the information
      x = shot['x_standardized']
      y = shot['y_standardized']
      # Set circle size
      # Set circle size
      circleSize = 2

      # Set color based on shot outcome
      if shot.shot_outcome == 'Goal':
          pitch.scatter(x, y, alpha=1, s=500, color='green', ax=ax)
          pitch.annotate(shot["lastName"], (x + 1, y - 2), ax=ax, fontsize=12)
      elif shot.shot_outcome == 'Accurate':
          pitch.scatter(x, y, alpha=1, s=500, color='blue', ax=ax)
          pitch.annotate(shot["lastName"], (x + 1, y - 2), ax=ax, fontsize=12)
      elif shot.shot_outcome == 'Missed':
          pitch.scatter(x, y, alpha=1, s=500, color='red', ax=ax)
          pitch.annotate(shot["lastName"], (x + 1, y - 2), ax=ax, fontsize=12)

  # Create legend
  goal_legend = plt.Line2D([0], [0], color="green", lw=4)
  accurate_legend = plt.Line2D([0], [0], color="blue", lw=4)
  missed_legend = plt.Line2D([0], [0], color="red", lw=4)
  ax.legend([goal_legend, accurate_legend, missed_legend], ['Goal', 'Accurate', 'Missed'], loc='upper right')

  # Set title
  match_name = plot_df['matchName'].iloc[0]  # Get the match name from the first row of plot_df
  fig.suptitle(match_name, fontsize=24)
  fig.set_size_inches(12, 8)
  plt.show()


TODO:
- Podzielic na polowy (1 i druga)